In [1]:
import sys, os
os.environ["CUDA_VISIBLE_DEVICES"] = "3" 

from flask import Flask, request
from transformers import (LlamaForCausalLM, CodeLlamaTokenizer,
                          BitsAndBytesConfig, StoppingCriteriaList)
import torch

from model_deployment.model_wrapper import CodeLLamaLocalWrapper
from model_deployment.codellama_utils import PeriodStoppingCriteria, do_beam_sample


/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from importlib import reload

In [25]:
import model_deployment.codellama_utils
reload(model_deployment.codellama_utils)
from model_deployment.codellama_utils import do_beam_sample

In [4]:
model_wrapper = CodeLLamaLocalWrapper.from_checkpoint("/home/ubuntu/coq-modeling/models/codellama-7b-basic/checkpoint-20500")

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.98s/it]


In [5]:
example_in = 'Lemma insertion_sort_perm: forall l, perm l (insertion_sort l).\n\tintros l.<THM-SEP>l: list nat\n\nperm l (insertion_sort l)'
example_out = '\n\tinduction l; simpl; unfold perm.'

In [5]:
example_in = """\
Lemma exists_min: forall (l : (list nat)), (l <> nil) -> exists h, min(l) = Some(h).
  induction l; simpl; intros.

  - contradiction.<THM-SEP>"""

In [32]:
example_in = """\
Lemma leb_refl: forall (n : nat), (n <=? n) = true.
Proof.
  induction n.<THM-SEP>

forall n1 n2 n3: nat,  
(n1 <=? n2) = true -> (n2 <=? n3) = true -> (n1 <=? n3) = true"""

In [ ]:
"""\
Lemma leb_refl: forall (n : nat), (n <=? n) = true.
  induction n.<THM-SEP>

(0 <=? 0) = true<GOAL-SEP>n: nat
IHn: (n <=? n) = true

(S n <=? S n) = true"""

In [33]:
collated_in = model_wrapper.collate_fn(example_in)
input_ids = model_wrapper.tokenizer(collated_in, return_tensors="pt")["input_ids"].to("cuda")

In [37]:
out = model_wrapper.model.generate(input_ids, max_new_tokens=32, temperature=0)

/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [38]:
model_wrapper.tokenizer.batch_decode(out, skip_special_tokenls=False)

['<s> Lemma leb_refl: forall (n : nat), (n <=? n) = true.\nProof.\n  induction n.<THM-SEP>\n\nforall n1 n2 n3: nat,  \n(n1 <=? n2) = true -> (n2 <=? n3) = true -> (n1 <=? n3) = true<TACTIC>\n  intros n1 n2 n3 H1 H2.</s>']

In [39]:
result = do_beam_sample(
    input_ids, 
    model_wrapper.model, 
    model_wrapper.tokenizer,
    6,
    6,
    model_wrapper.stopping_criteria,
    batch_size=3,
    )

['\n', '', ' ', '(*', '   ', '  ']
['\n ', '\n\n', '\nAb', '\nint', '\n  ', '\n   ', '\n', ' \n', '   \n', '  \n', ' (*', '  (*', '(* ->', '(* n', '(* ', '(* ref', '(* induction', '(*n', '   (*', '    (*', ' induction', ' (', ' auto', ' Ab', '  (', '  ref', '  //', '  ->', '    (', '   (', '    Ab', '    //', '    ref', '   //', '   Ab', '   ref']
['\n  intros', '\n\n ', '\nAbort', '\nintros', '\n  (*', '\n  \n', '\n\n   ', '\n  -', '\n  (', '\n  intro', '\n\nAb', '\n    intros', '\n  ref', '\n\n(*', '\n    (*', '\n\nint', '\n\n  ', '\n   \n', '\nintu', '\n    intro', '\n    (', '\n   (*', '\n    simpl', '\n   intros', '\nAbbre', '\n   (', '\n  2', '\n  1', '\nintroduction', '\nintrol', '\nintuit', '\nAbs', '\nintrov', '\nAborted', '\nAbbr', '\nAbORT']
['\n  intros n', '\nAbort.', '\n  intros.', '\n\n  intros', '\n\n  -', '\nintros.', '\nintros n', '\n  \n ', '\n  intros;', '\n\n  (*', '\n\n  +', '\n  (* induction', '\n\n  intro', '\n\n  simpl', '\n  \n   ', '\n  intros m', '\n  (* n',

In [27]:
result.tactics

['', '\n\n  - ', '\n\n\n  - ', '', '\n  - ', '\n\n  -s']

In [20]:
list(zip(result.scores, result.tactics))

[(-4.127943515777588, '\n\tdestruct l.'),
 (-1.390378475189209, '\n\tinduction l.'),
 (-3.5746660232543945, '\n\tunfold perm.'),
 (-9.438248634338379, '\n\tapply insertion.'),
 (-11.594498634338379, '\n\tapply insertion')]

In [60]:
input_ids.expand(2, -1)

tensor([[    1, 11894,  4635,   291, 29918,  6605, 29918, 17858, 29901, 25345,
           301, 29892,  3635,   301,   313,  7851,   291, 29918,  6605,   301,
           467,    13,    12,   524,  1883,   301, 19423,  4690, 29924, 29899,
          1660, 29925, 29958, 29880, 29901,  1051, 14033,    13,    13, 17858,
           301,   313,  7851,   291, 29918,  6605,   301, 29897,    13, 29966,
          6040,  1783,  2965, 29958,    13],
        [    1, 11894,  4635,   291, 29918,  6605, 29918, 17858, 29901, 25345,
           301, 29892,  3635,   301,   313,  7851,   291, 29918,  6605,   301,
           467,    13,    12,   524,  1883,   301, 19423,  4690, 29924, 29899,
          1660, 29925, 29958, 29880, 29901,  1051, 14033,    13,    13, 17858,
           301,   313,  7851,   291, 29918,  6605,   301, 29897,    13, 29966,
          6040,  1783,  2965, 29958,    13]], device='cuda:0')

In [61]:
prepared_inputs = model_wrapper.model.prepare_inputs_for_generation(input_ids.expand(2, -1))

In [62]:
out = model_wrapper.model(**prepared_inputs)

In [64]:
out[1][0][1].shape

torch.Size([2, 32, 55, 128])

In [53]:
out[1]

((tensor([[[[ 5.5420e-01, -6.6699e-01,  3.8403e-01,  ..., -5.7526e-02,
              2.9272e-01, -7.4036e-02],
            [ 1.0382e-01, -2.2815e-01,  2.6184e-02,  ...,  4.8608e-01,
             -3.2593e-01,  4.4897e-01],
            [-1.6736e-01, -6.9580e-02, -2.5342e-01,  ...,  4.5581e-01,
             -3.1909e-01,  4.5654e-01],
            ...,
            [-4.8981e-02,  3.6060e-01, -5.5859e-01,  ...,  5.5029e-01,
             -4.3872e-01,  6.1719e-01],
            [-2.2461e-01, -6.2256e-02, -3.0273e-01,  ...,  9.9121e-02,
              9.6252e-02,  2.8906e-01],
            [ 7.3914e-02, -1.8652e-01,  2.8760e-01,  ...,  3.1885e-01,
             -4.3164e-01,  3.9258e-01]],
  
           [[ 5.2783e-01,  6.0693e-01,  3.8574e-01,  ...,  1.1943e+00,
             -8.7256e-01,  9.8633e-01],
            [ 1.7041e-01,  5.8899e-02, -9.6558e-02,  ...,  5.0087e-03,
              1.1633e-01, -8.7952e-02],
            [-1.7114e-01, -1.3965e-01,  2.6688e-02,  ..., -1.1890e-01,
              2.8809

In [48]:
result.scores

[-3.576564073562622,
 -1.397013545036316,
 -4.399918079376221,
 -4.137808799743652,
 -4.5787553787231445]

In [5]:
model_inputs = model_wrapper.model.prepare_inputs_for_generation(input_ids)

In [9]:
reversed_vocab = dict((v, k) for k, v in model_wrapper.tokenizer.get_vocab().items())

In [15]:
logits = model_wrapper.model(**model_inputs)

/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


In [18]:
model_wrapper.model._use_cache

AttributeError: 'LlamaForCausalLM' object has no attribute '_use_cache'

In [7]:
model_inputs["input_ids"].shape

torch.Size([1, 55])

In [20]:
model_output = model_wrapper.model(**model_inputs)

In [24]:
model_output[0].shape

torch.Size([1, 55, 32016])

In [47]:
len(model_output[1])

32

In [46]:
model_output[1][4][1].shape

torch.Size([1, 32, 55, 128])

In [ ]:
def do_beam_sample(
    input_ids: torch.LongTensor,
    model: LlamaForCausalLM,
    tokenizer: CodeLlamaTokenizer,
    n_recs: int,
    period_stopping: PeriodStoppingCriteria,
    batch_size: int = 2,
) -> SampleResult:
    past = None
    while True:
        prepared_input = model.prepare_inputs_for_generation(input_ids, past=past)
        model_output = model(**prepared_input)
        model_logits = model_output[0]
        past = model_output[1]
        next_token_logits = model_logits[:, -1, :]




In [8]:
pad_token = "<PRE>" 
encoded_ids = tokenizer.encode(pad_token)
assert len(encoded_ids) == 2
assert encoded_ids[0] == tokenizer.bos_token_id

tokenizer.pad_token = pad_token
tokenizer.pad_token_id = encoded_ids[1] 

In [9]:
test_in = 'Lemma exists_min: forall (l : (list nat)), \n    (l <> nil) -> exists h, min(l) = Some(h).\nProof. \nintros l H. <FILL_ME>' 
test_out = "\n    + "

In [65]:
input_ids = tokenizer(test_in, return_tensors="pt")["input_ids"].to("cuda")
period_stopping.set_num_periods(input_ids)
stopping_list = StoppingCriteriaList([period_stopping])

In [89]:
model_output = model.generate(
    input_ids, 
    temperature=1,
    do_sample=True,
    max_new_tokens=200, 
    output_scores=True,
    return_dict_in_generate=True,
    stopping_criteria=stopping_list,
    ) 
single_output = model_output.sequences[0].to("cpu")
token_output = tokenizer.decode(single_output[input_ids.shape[1]:], skip_special_tokens=True)
token_output


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'\ndestruct l as [| l0 l1].'

In [90]:
type(model_output)

transformers.generation.utils.SampleDecoderOnlyOutput

In [67]:
tokenizer.batch_decode(model_output.sequences)

['<s> Lemma exists_min: forall (l : (list nat)), \n    (l <> nil) -> exists h, min(l) = Some(h).\nProof. \nintros l H. \napply exists_min_nonempty in H.']

In [92]:
tokenizer.decode(model_output.sequences[0, input_ids.shape[1]:], skip_special_tokens=True)

'\ndestruct l as [| l0 l1].'

In [80]:
model_output.scores

(tensor([[   -inf,    -inf, 10.8047,  ...,    -inf,    -inf,    -inf]],
        device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[  -inf,   -inf, 9.7266,  ...,   -inf,   -inf,   -inf]],
        device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'))

In [81]:

def get_sequence_score(input_sequence: torch.LongTensor,
                       whole_sequence: torch.LongTensor, 
                       scores: tuple[torch.FloatTensor],
                       stop_criteria: PeriodStoppingCriteria) -> float:
    assert len(scores) == int(whole_sequence.shape[0] - input_sequence.shape[0])
    sequence_score = 0
    start_idx = whole_sequence.shape[0] - len(scores)
    stop_criteria.set_num_periods(input_sequence[None, :])
    for i in range(len(scores)):
        index = whole_sequence[start_idx + i] 
        score_at_i = scores[i][0, index] - torch.logsumexp(scores[i][0], axis=0)
        sequence_score += (score_at_i)
        if stop_criteria(whole_sequence[None, :(start_idx + i + 1)], scores):
            break
    return sequence_score



In [86]:
get_sequence_score(
    input_ids[0], 
    model_output.sequences[0], 
    model_output.scores, period_stopping)

tensor(-5.2211, device='cuda:0')

In [36]:
model_output.scores[1].shape

torch.Size([1, 32016])

In [37]:
len(model_output.scores)

8

In [32]:
model_output

SampleDecoderOnlyOutput(sequences=tensor([[    1, 11894,  4864, 29918,  1195, 29901, 25345,   313, 29880,   584,
           313,  1761, 14033,  8243, 29871,    13,  1678,   313, 29880, 15271,
          4263, 29897,  1599,  4864,   298, 29892,  1375, 29898, 29880, 29897,
           353,  3834, 29898, 29882,   467,    13, 28116, 29889, 29871,    13,
           524,  1883,   301,   379, 29889, 29871,    13,  7854,  1247,   313,
          1195, 29898, 29880,  8106]], device='cuda:0'), scores=(tensor([[   -inf,    -inf, 10.8047,  ...,    -inf,    -inf,    -inf]],
       device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[  -inf,   -inf, 7.7188,  ...,   -inf,   -inf,   -inf]],
       device='cuda:0'), tensor([[  -inf,   -inf, 8.1875,  ...,   -inf,   -inf,   -inf]],
       device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[  -inf,   -inf, 7.5156,  ...,   -inf,   -inf,   -inf]],
       device='cuda:0')

In [31]:
model_output.scores

(tensor([[   -inf,    -inf, 10.8047,  ...,    -inf,    -inf,    -inf]],
        device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[  -inf,   -inf, 7.7188,  ...,   -inf,   -inf,   -inf]],
        device='cuda:0'),
 tensor([[  -inf,   -inf, 8.1875,  ...,   -inf,   -inf,   -inf]],
        device='cuda:0'),
 tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'),
 tensor([[  -inf,   -inf, 7.5156,  ...,   -inf,   -inf,   -inf]],
        device='cuda:0'),
 tensor([[   -inf,    -inf, 10.0156,  ...,    -inf,    -inf,    -inf]],
        device='cuda:0'),
 tensor([[  -inf,   -inf, 9.2188,  ...,   -inf,   -inf,   -inf]],
        device='cuda:0'))

In [20]:
period_stopping.set_num_periods(input_ids)

In [8]:
period_stopping.num_input_periods

tensor(2, device='cuda:0')

In [19]:
tokenizer.batch_decode(model_output.sequences)

['<s> Lemma exists_min: forall (l : (list nat)), \n    (l <> nil) -> exists h, min(l) = Some(h).\nProof. \nintros l H. \ndestruct l as [|h t].</s>',
 "<s> Lemma exists_min: forall (l : (list nat)), \n    (l <> nil) -> exists h, min(l) = Some(h).\nProof. \nintros l H. \ndestruct l as [|x l'].</s>",
 '<s> Lemma exists_min: forall (l : (list nat)), \n    (l <> nil) -> exists h, min(l) = Some(h).\nProof. \nintros l H. \ndestruct l as [|x xs].</s>',
 '<s> Lemma exists_min: forall (l : (list nat)), \n    (l <> nil) -> exists h, min(l) = Some(h).\nProof. \nintros l H. \ndestruct l as [| h t].</s>',
 "<s> Lemma exists_min: forall (l : (list nat)), \n    (l <> nil) -> exists h, min(l) = Some(h).\nProof. \nintros l H. \ndestruct l as [| x l'].</s>"]

In [ ]:
#collated_in = collate_input(test_in)
prompt = '''def remove_non_ascii(s: str) -> str:
    """ <FILL_ME>
    return result
'''

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")

In [ ]:
output = model.generate(input_ids, max_new_tokens=200)
output = output[0].to("cpu")

In [ ]:
only_output = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)

In [ ]:
2 in output

In [ ]:
tokenizer.eos_token_id

In [ ]:
print(only_output)